# Reservoir operation modelling
In this Notebook we will see how we can simulate the operation of water supply reservoir system.

<left><img src="images/Dam1.gif" width = "800px"><left>
    
We consider a simple illustrative system where a reservoir is operated to supply water to a domestic consumption node, while ensuring a minimum environmental flow in the downstream river (also called “environmental compensation flow”) and maintaining the water level in the reservoir within prescribed limits. We use a mathematical model to link all the key variables that represent the reservoir dynamics (inflow, storage and outflows) and use model simulation/optimisation to determine the reservoir release scheduling that will best meet the water demand over a coming period of time, given the predicted (or assumed) scenario of future inflows.
<left> <img src="images/system_representation_IO0.png" width = "600px"><left>
## Import libraries
Before getting started, let's import some libraries that will be used throughout the Notebook:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact # to create interactive elements and figures

## The reservoir model

The mathematical model of the reservoir essentially consists of a water balance equation, where the storage (***s***) at a future time step (for example, at the beginning of the next week) is predicted from the storage at the current time (the beginning of the this week) by adding and subtracting the inflows and outflows that will occur during the temporal interval ahead:

$s(t+1) = s(t) + inflow(t) – evap(t) – env(t) - spill(t) – supply(t)$   

Where

***s(t)*** = reservoir storage at time-step t, in Vol (for example: ML)

***inflow(t)*** = reservoir inflows in the interval [t,t+1], in Vol/time (for example: ML/week).

***evap(t)*** = evaporation from the reservoir surface area in the interval [t,t+1], in Vol/time (for example: ML/week).

***env(t)*** = environmental compensation flow in the interval [t,t+1], in Vol/time (for example: ML/week). 

***spill(t)*** = outflow through spillways (if any) in the interval [t,t+1], in Vol/time (for example: ML/week). 

***supply(t)*** = regulated reservoir release for water supply in the interval [t,t+1], in Vol/time (for example: ML/week). 

<left> <img src="images/system_representation_IO1.png" width = "600px"><left>
    
#### Implementation of the reservoir simulation function
Here we define a function that implements the reservoir simulation, that is, iteratively apply the mass balance equation and reconstruct the temporal evolution of the reservoir variables over the simulation period

In [ ]:
def res_sim(inflow,evap,demand,s_0,s_max,env_min,supply):
    """
    This is a model that simulates the operation of a single reservoir system. 
    It essentially consists of a water balance equation, 
    where the storage (s) at a future time step is predicted from the storage at the current time 
    by adding and subtracting the inflows and outflows that will occur during the temporal interval ahead

    The inputs of the model are:

    inflow = time series of reservoir inflows [ML]

    evap = time series of evaporation from the reservoir surface area [ML]
    
    demand = time series of water demand [ML]
    
    s_0 = initial reservoir storage [ML]
    
    s_max = maximum storage capacity of the reservoir [ML]

    env_min = minimum environmental flow [ML]
    
    supply = regulated reservoir release for water supply [ML]
    
    And the outpus are:
    
    s = reservoir storage [ML]
    
    env = environmental compensation flow [ML]
    
    spill = outflow through spillways [ML]
    
    supply = regulated reservoir release for water supply [ML]
    
    """
    
    # Please write the code of the model here
    
    
    return s,env,spill,supply

#### Definition of inflow and demand scenarios
Let's assume we want to look at the next 8 weeks the number of weeks, and assume we have forecasts of inflows, evaporation and demand for this period.

In [ ]:
inflow = np.array([15,17,19,11,9,4,3,8]) # (ML/week) time series of inflow forecasts
evap = np.array([1,1,2,2,2,2,2,3]) # (ML/week) time series of evaporation forecasts
demand = np.array([13,13,17,18,20,22,25,26]) # (ML/week) time series of demand forecasts

Plot the **inflow** forecast:

Plot the **demand** forecast:

#### Definition of other input parameters
Let's define other variables that are needed for the reservoir system simulation, such as the reservoir storage capacity, the environmental compensation flow, etc.

In [ ]:
### Constraints ###
s_max = 150 #  (ML) Maximum storage (=reservoir capacity)
env_min = 2 # (ML/week)   # Environmental compensation flow

### Initial conditions ###
s_0 = 80 # (ML) # Storage volume at the beginning of the simulation period

## Determining the release scheduling by trial and error (manual optimisation)
Here we want to use the reservoir model to assist the reservoir operator in determining the best scheduling of regulated reservoir releases (`supply`) in response to a certain scenario of inflows. The goal is to minimise the deficit with respect to a prescribed water demand, that is, to minimise the objective function:

$$TSD = \sum_{t=1}^{T} [ \ max( \ 0, \ demand(t)-supply(t) \ ) \ ]^2 $$

where `T` is the length of the simulation period that we are considering, and `demand` is the water demand for each time-interval in that period, and `TSD` stands for **Total Squared Deficit**. Notice that the function $max(0,...)$ enables us to only count the difference between `demand` and `supply` when this is positive, that is, when the `supply` is smaller than the `demand`, and a water shortage is indeed produced. Also, the squaring is a 'mathematical trick' to make sure that larger deficit amounts are given more weight than smaller ones. This translates the fact that small deficit amounts are easier to mitigate and hence more acceptable, while larger ones can cause disproportionately severe impacts and should be avoided as much as possible.

In [ ]:
def TSD_func():


#### Determining the optimal release scheduling via interactive visualisation

Use the slider to set the release amount for each week in the simulation period, and in doing so try to minimise the Total Squared Deficit. 

In [ ]:
@interact()
def int_res_operation_1():


**Comment**: clearly it is not possible to fully meet the demand at all times in the simulation period. For example if we fully cover the demand for the first 7 weeks, we drawdown the reservoir to a point that we are forced to dramatically reduce the release in the last week, causing a very severe deficit. A more effective approach is to cause smaller deficits across all time steps, that is, tolerate some small deficits even when in principle we may fully cover the demand, in order to prevent more severe deficits in the later period. This type of approach is called ***hedging*** (see for example [You and Cai 2008](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2006WR005481)).

Btw, the minimum TSD value that can be achieved is **49, try to beat it!**

## From single to multi-objective
Now let's assume that, besides minimising supply deficits, we are also interested in minimising the chances that the reservoir level go below a minimum threshold. This could be, for example, because the quality of the water deteriorates when levels are low, requiring more costly treatment. We measure how well this criterion is satisfied by the following objective function:

$$TSV = \sum_{t=1}^{T} max ( \ smin - s(t) , \ 0)$$

where, again, `T` is the length of the simulation period, `s` is the reservoir storage, and `s_min` is the minimum reservoir storage threshold that should preferably not be transpassed (`TSV` stands for **Total Storage Violation**). 

In [ ]:
def TSV_func():


For our case, let's set this threshold to 30 ML:

In [ ]:
# Minimum storage threshold
s_min = 30 # in ML

Now use the slider to set the release amount for each week in the simulation period, in a way that jointly minimise the Total Squared Deficit and the Minimum Storage Violation. 

In [ ]:
@interact()
def int_res_operation_2():


***Comment*** It is possible to find a release scheduling that produce no violation of the minimum storage threshold, although it will produce some supply deficit - the record is **305, can you beat it?**. However, one could also allow some violation of the storage threshold in order to reduce the deficits. The two objectives are conflicting: improving on one of them implies doing worse on the other.

### References 

Deb K. et al (2002) A fast and elitist multiobjective genetic algorithm: NSGA-II, IEEE Transactions on Evolutionary Computation, 6(2), 182-197, doi:10.1109/4235.996017.

You J.Y. and Cai X. (2008) Hedging rule for reservoir operations: 1. A theoretical analysis, Water Resour. Res., 44, W01415, doi:10.1029/2006WR005481.